In [1]:
from netCDF4 import Dataset
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, ALL_TIMES,interpline, CoordPair, xy_to_ll, ll_to_xy)
import pandas as pd
import numpy as np
import os
import shutil
import re
import netCDF4 as nc
import xarray as xr

In [2]:
#mask out seas-1620

ds_lu_1620 = xr.open_dataset('wrfout_d04_2016-06-30_22:00:00')
lu_1620 = ds_lu_1620['LU_INDEX']

lu_1620 = lu_1620[0,:,:]

del lu_1620.coords['XTIME']

lu_1620_df = lu_1620.to_dataframe().reset_index()

In [4]:
#mask out seas-2040s

ds_lu_2040s = xr.open_dataset('wrfout_d04_2040-06-18_23:00:00')
lu_2040s = ds_lu_2040s['LU_INDEX']

lu_2040s = lu_2040s[0,:,:]

del lu_2040s.coords['XTIME']

lu_2040s_df = lu_2040s.to_dataframe().reset_index()

# 2010s

In [5]:
ds1 = xr.open_dataset('ssp245_t2_1120_era5_corrected_dailymax.nc')
t2_1120_ssp245_dailymax = ds1['ssp245_t2_1120_era5_corrected_dailymax']

t2_1120_ssp245_dailymax_df = t2_1120_ssp245_dailymax.to_dataframe().reset_index()

#Use 33℃ as a threshold

t2_1120_dailymax_thres = (t2_1120_ssp245_dailymax_df[t2_1120_ssp245_dailymax_df['ssp245_t2_1120_era5_corrected_dailymax']>=33]).reset_index(drop=True)

In [6]:
t2_1120_dailymax_thres = t2_1120_dailymax_thres.sort_values(['XLAT','XLONG','Time'], ascending=True)


t2_1120_dailymax_thres_latlon = t2_1120_dailymax_thres.groupby(['XLAT','XLONG'])


t2_1120_dailymax_thres['auxi_col'] = t2_1120_dailymax_thres_latlon['Time'].rank()


t2_1120_dailymax_thres['date_sub'] = t2_1120_dailymax_thres['Time'] - pd.to_timedelta(t2_1120_dailymax_thres['auxi_col'],unit='d')

In [7]:

HWD_1120 = t2_1120_dailymax_thres.groupby(['XLAT','XLONG','south_north','west_east','date_sub']).count().reset_index()
HWD_1120 = HWD_1120.rename(columns={'auxi_col':'concecutive_hot_days_num'})

In [8]:

HWD_1120 = HWD_1120[HWD_1120['concecutive_hot_days_num']>=3]

In [9]:

HWD_1120 = HWD_1120.groupby(['XLAT','XLONG','south_north','west_east'])['concecutive_hot_days_num'].mean().reset_index()

In [10]:
#mask out the sea

HWD_1120 = pd.merge(HWD_1120,lu_1620_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

HWD_1120['concecutive_hot_days_num'] = HWD_1120['concecutive_hot_days_num'].fillna(0)

HWD_1120.loc[HWD_1120.LU_INDEX == 16, 'concecutive_hot_days_num'] = 9999

HWD_1120 = HWD_1120.rename(columns={"concecutive_hot_days_num": "hwd"})

In [13]:
HWD_1120.to_csv('HWD_1120.csv',index=False,columns=['hwd','XLAT','XLONG'])

HWD_1120.to_csv('HWD_1120_withgrids.csv',index=False,columns=['XLAT','XLONG','hwd','south_north','west_east'])

# 2040s

In [14]:
ds1 = xr.open_dataset('ssp245_t2_4049_era5_corrected_dailymax.nc')
t2_4049_ssp245_dailymax = ds1['ssp245_t2_4049_era5_corrected_dailymax']

t2_4049_ssp245_dailymax_df = t2_4049_ssp245_dailymax.to_dataframe().reset_index()

#Use 33℃ as a threshold

t2_4049_dailymax_thres = (t2_4049_ssp245_dailymax_df[t2_4049_ssp245_dailymax_df['ssp245_t2_4049_era5_corrected_dailymax']>=33]).reset_index(drop=True)

In [15]:
t2_4049_dailymax_thres = t2_4049_dailymax_thres.sort_values(['XLAT','XLONG','Time'], ascending=True)


t2_4049_dailymax_thres_latlon = t2_4049_dailymax_thres.groupby(['XLAT','XLONG'])


t2_4049_dailymax_thres['auxi_col'] = t2_4049_dailymax_thres_latlon['Time'].rank()


t2_4049_dailymax_thres['date_sub'] = t2_4049_dailymax_thres['Time'] - pd.to_timedelta(t2_4049_dailymax_thres['auxi_col'],unit='d')

In [16]:

HWD_4049 = t2_4049_dailymax_thres.groupby(['XLAT','XLONG','south_north','west_east','date_sub']).count().reset_index()
HWD_4049 = HWD_4049.rename(columns={'auxi_col':'concecutive_hot_days_num'})

In [17]:

HWD_4049 = HWD_4049[HWD_4049['concecutive_hot_days_num']>=3]

In [18]:

HWD_4049 = HWD_4049.groupby(['XLAT','XLONG','south_north','west_east'])['concecutive_hot_days_num'].mean().reset_index()

In [19]:
#mask out the sea

HWD_4049 = pd.merge(HWD_4049,lu_2040s_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

HWD_4049['concecutive_hot_days_num'] = HWD_4049['concecutive_hot_days_num'].fillna(0)

HWD_4049.loc[HWD_4049.LU_INDEX == 16, 'concecutive_hot_days_num'] = 9999

HWD_4049 = HWD_4049.rename(columns={"concecutive_hot_days_num": "hwd"})

In [20]:
HWD_4049.to_csv('HWD_4049.csv',index=False,columns=['hwd','XLAT','XLONG'])

HWD_4049.to_csv('HWD_4049_withgrids.csv',index=False,columns=['XLAT','XLONG','hwd','south_north','west_east'])

# 2090s

In [21]:
ds1 = xr.open_dataset('ssp245_t2_9099_era5_corrected_dailymax.nc')
t2_9099_ssp245_dailymax = ds1['ssp245_t2_9099_era5_corrected_dailymax']

t2_9099_ssp245_dailymax_df = t2_9099_ssp245_dailymax.to_dataframe().reset_index()

#Use 33℃ as a threshold

t2_9099_dailymax_thres = (t2_9099_ssp245_dailymax_df[t2_9099_ssp245_dailymax_df['ssp245_t2_9099_era5_corrected_dailymax']>=33]).reset_index(drop=True)

In [22]:
t2_9099_dailymax_thres = t2_9099_dailymax_thres.sort_values(['XLAT','XLONG','Time'], ascending=True)


t2_9099_dailymax_thres_latlon = t2_9099_dailymax_thres.groupby(['XLAT','XLONG'])


t2_9099_dailymax_thres['auxi_col'] = t2_9099_dailymax_thres_latlon['Time'].rank()


t2_9099_dailymax_thres['date_sub'] = t2_9099_dailymax_thres['Time'] - pd.to_timedelta(t2_9099_dailymax_thres['auxi_col'],unit='d')

In [23]:

HWD_9099 = t2_9099_dailymax_thres.groupby(['XLAT','XLONG','south_north','west_east','date_sub']).count().reset_index()
HWD_9099 = HWD_9099.rename(columns={'auxi_col':'concecutive_hot_days_num'})

In [24]:

HWD_9099 = HWD_9099[HWD_9099['concecutive_hot_days_num']>=3]

In [25]:

HWD_9099 = HWD_9099.groupby(['XLAT','XLONG','south_north','west_east'])['concecutive_hot_days_num'].mean().reset_index()

In [26]:
#mask out the sea

HWD_9099 = pd.merge(HWD_9099,lu_2040s_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

HWD_9099['concecutive_hot_days_num'] = HWD_9099['concecutive_hot_days_num'].fillna(0)

HWD_9099.loc[HWD_9099.LU_INDEX == 16, 'concecutive_hot_days_num'] = 9999

HWD_9099 = HWD_9099.rename(columns={"concecutive_hot_days_num": "hwd"})

In [28]:
HWD_9099.to_csv('HWD_9099.csv',index=False,columns=['hwd','XLAT','XLONG'])

HWD_9099.to_csv('HWD_9099_withgrids.csv',index=False,columns=['XLAT','XLONG','hwd','south_north','west_east'])

# diff

In [2]:
HWD_1120 = pd.read_csv('HWD_1120_withgrids.csv')

HWD_1120 = HWD_1120.replace(9999,np.nan)

HWD_1120 = HWD_1120.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [3]:
HWD_4049 = pd.read_csv('HWD_4049_withgrids.csv')

HWD_4049 = HWD_4049.replace(9999,np.nan)

HWD_4049 = HWD_4049.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [4]:
HWD_9099 = pd.read_csv('HWD_9099_withgrids.csv')

HWD_9099 = HWD_9099.replace(9999,np.nan)

HWD_9099 = HWD_9099.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [5]:
#test

print((HWD_1120['west_east']-HWD_4049['west_east']).min())
print((HWD_1120['west_east']-HWD_4049['west_east']).max())
print((HWD_1120['south_north']-HWD_4049['south_north']).max())
print((HWD_1120['south_north']-HWD_4049['south_north']).min())
print((HWD_4049['west_east']-HWD_9099['west_east']).max())
print((HWD_4049['west_east']-HWD_9099['west_east']).min())
print((HWD_4049['south_north']-HWD_9099['south_north']).max())
print((HWD_4049['south_north']-HWD_9099['south_north']).min())

0
0
0
0
0
0
0
0


In [6]:
HWD_diff_2040s = HWD_1120.copy()
HWD_diff_2090s = HWD_1120.copy()

In [7]:
HWD_diff_2040s['diff'] = HWD_4049['hwd']-HWD_1120['hwd']

HWD_diff_2090s['diff'] = HWD_9099['hwd']-HWD_1120['hwd']

In [8]:
HWD_diff_2040s['diff'] = HWD_diff_2040s['diff'].fillna(9999)

HWD_diff_2090s['diff'] = HWD_diff_2090s['diff'].fillna(9999)

In [36]:
HWD_diff_2040s.to_csv('HWD_diff_2040s.csv')
HWD_diff_2090s.to_csv('HWD_diff_2090s.csv')